Model

In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#checking for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cpu


In [4]:
transformer = transforms.Compose([ 
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),  
    transforms.ToTensor(), 
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5]) 
])

In [6]:
#path for training and testing
train_path = 'D://program//pytorch//sportsball/dataset/train'
test_path = 'D://program//pytorch//sportsball/dataset/test'

In [7]:
#dataloade
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size = 64, shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size = 32, shuffle=True
)

In [8]:
#category
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['american_football', 'baseball', 'basketball', 'billiard_ball', 'bowling_ball', 'cricket_ball', 'football', 'golf_ball', 'hockey_ball', 'hockey_puck', 'rugby_ball', 'shuttlecock', 'table_tennis_ball', 'tennis_ball', 'volleyball']


In [16]:
print(count)

NameError: name 'count' is not defined

In [9]:
#cnn network
class ConvNet(nn.Module):
    def __init__(self, num_classes = 15):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 3,out_channels =12,kernel_size=3,stride=1,padding=1) 
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels= 12,out_channels =20,kernel_size=3,stride=1,padding=1) 
        self.relu2 = nn.ReLU() 
        self.conv3 = nn.Conv2d(in_channels= 20,out_channels =32,kernel_size=3,stride=1,padding=1) 
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()  
        self.fc =nn.Linear(in_features=75 * 75* 32 ,out_features=num_classes)

        #feed forward function

    def forward(self,input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)

            #Above output will be in matrix  form (256,32,75,75)

        output= output.view(-1,32*75*75)
        output = self.fc(output)

        return output

In [10]:
model =ConvNet(num_classes=6).to(device)

In [11]:
#Optimizer and loss function

optimizer = Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function =nn.CrossEntropyLoss()

In [12]:
num_epochs = 10

In [13]:
#calculate the size of training and testing images

train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

In [14]:
print(train_count,test_count)

7328 1841


In [15]:
#model training and saving best model
best_accuracy = 0.0
for epoch in range(num_epochs):

    #evaluation and training on training dataset
    model.train()
    train_accuracy =0.0
    train_loss = 0.0

    for i , (images,labels) in enumerate (train_loader):
        if torch.cuda.is_available():
            images= Variable(images.cuda())
            labels=Variable(labels.cuda())
        optimizer.zero_grad()  #backprop accumulates gradients, and not want them in minibatches
        outputs = model(images)

        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step() #update weight and bias

        train_loss += loss.cpu().data*images.size(0)    
        _,prediction = torch.max(outputs.data,1)   
        train_accuracy +=int(torch.sum(prediction == labels.data))
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count

    #evalutaion on testing dataset

    model.eval()

    test_accuracy =0.0
    for i , (images,labels) in enumerate (test_loader):
        if torch.cuda.is_available():
            images= Variable(images.cuda())
            labels=Variable(labels.cuda())
        
        outputs=model(images)
        _,prediction = torch.max(outputs.data,1)   
        test_accuracy +=int(torch.sum(prediction == labels.data))
    test_accuracy = test_accuracy/test_count

    print('Epoch :' +str(epoch)+'Train loss:'+str(int(train_loss))+'Train accuracy:'+str(train_accuracy)+'Test accuracy:'+str(test_accuracy))

    #save the best model

    if test_accuracy >best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy


IndexError: Target 6 is out of bounds.

Inference - testing on datas

In [5]:
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
from PIL import Image
import cv2

In [18]:
train_path = 'D://program//pytorch//intel//scene_detection//seg_train//seg_train'
pred_path = 'D://program//pytorch//intel//scene_detection//seg_pred//seg_pred'

In [19]:
#category
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [20]:
class ConvNet(nn.Module):
    def __init__(self, num_classes = 6):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 3,out_channels =12,kernel_size=3,stride=1,padding=1) 
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels= 12,out_channels =20,kernel_size=3,stride=1,padding=1) 
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels= 20,out_channels =32,kernel_size=3,stride=1,padding=1) 
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()    
        self.fc =nn.Linear(in_features=75 * 75* 32 ,out_features=num_classes)
    def forward(self,input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output= output.view(-1,32*75*75)
        output = self.fc(output)

        return output

In [21]:
checkpoint =torch.load('best_checkpoint.model')
model = ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [22]:
transformer = transforms.Compose([ #preprocessing
    transforms.Resize((150,150)),
    transforms.ToTensor(), #0-225 to 0to 1, numpy to tensor  change pixel range
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5]) # 0 to 1 to -1 to 1  , formua (x-mean)/std
])

In [29]:
#prediction fuction
def prediction(img_path,transformer):
    image = Image.open(img_path)
    image_tensor = transformer(image).float()
    image_tensor =image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()
    input = Variable(image_tensor)

    output = model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    return pred



In [30]:
images_path = glob.glob(pred_path+'/*.jpg')

In [31]:
pred_dict={} #image name as key , value as prediction
for i in images_path:
    pred_dict[i[i.rfind('/')+1:]] = prediction(i,transformer)

In [32]:
pred_dict

{'seg_pred\\10004.jpg': 'street',
 'seg_pred\\10005.jpg': 'mountain',
 'seg_pred\\10012.jpg': 'street',
 'seg_pred\\10013.jpg': 'glacier',
 'seg_pred\\10017.jpg': 'mountain',
 'seg_pred\\10021.jpg': 'forest',
 'seg_pred\\1003.jpg': 'sea',
 'seg_pred\\10034.jpg': 'glacier',
 'seg_pred\\10038.jpg': 'sea',
 'seg_pred\\10040.jpg': 'street',
 'seg_pred\\10043.jpg': 'sea',
 'seg_pred\\10045.jpg': 'street',
 'seg_pred\\10047.jpg': 'mountain',
 'seg_pred\\10048.jpg': 'buildings',
 'seg_pred\\10052.jpg': 'buildings',
 'seg_pred\\10054.jpg': 'glacier',
 'seg_pred\\10059.jpg': 'forest',
 'seg_pred\\10060.jpg': 'forest',
 'seg_pred\\10066.jpg': 'mountain',
 'seg_pred\\10069.jpg': 'sea',
 'seg_pred\\10073.jpg': 'street',
 'seg_pred\\10079.jpg': 'street',
 'seg_pred\\1008.jpg': 'street',
 'seg_pred\\10082.jpg': 'street',
 'seg_pred\\10083.jpg': 'glacier',
 'seg_pred\\10090.jpg': 'buildings',
 'seg_pred\\10092.jpg': 'forest',
 'seg_pred\\10096.jpg': 'mountain',
 'seg_pred\\101.jpg': 'glacier',
 'seg_